# 图

## 1.图的抽象数据类型

1.Graph()创建一个空<br>
2.addVertex(vert)向图中添加一个顶点实例<br>
3.addEdge(fromVert, toVert)向图中添加一条有向边用于链接顶点fromVert,toVert<br>
4.addEdge(fromVert, toVert, weight)向图中添加一条带权重weight的有向边用于链接顶点fromVert,toVert<br>
5.getVertex(vertKey)在图中找到名为vertKey的顶点<br>
6.getVertices()以列表的形式返回图中的所有顶点。<br>
7.通过in语句判断顶点是否存在

## 2.图的实现

1.邻接矩阵<br>
2.邻接表

### 2.1定义顶点

In [1]:
class Vertex:
    def __init__(self, key, color = "white", preNode = None, distance = None):
        #顶点键
        self.key = key
        #邻接表(顶点：权重)
        self.connectedTo = {}
        #节点状态
        self.color = color
        #前驱节点
        self.preNode = preNode
        #与原点的距离
        self.distance = distance
        
    def getKey(self):
        return self.key
        
    def setColor(self, color):
        self.color = color
        
    def getColor(self):
        return self.color
    
    def setPreNode(self, node):
        self.preNode = node
        
    def getPreNode(self):
        return self.preNode
    
    def setDistance(self, distance):
        self.distance = distance
        
    def getDistance(self):
        return self.distance
        
    def __str__(self):
        return str(self.key)+"connectedTo"+str([x.key for x in self.connectedTo])
    
    def addNeighbor(self, neighbor, weight):
        self.connectedTo[neighbor] = weight
        
    def getConnections(self):
        return self.connectedTo.keys()
    
    def getWeight(self, nbr):
        return self.connectedTo[nbr]

### 2.2定义图

In [2]:
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0
        
    def __contains__(self, n):
        return n in self.vertList
    
    def __iter__(self):
        return iter(self.vertList.values())
    
    def addVertex(self, key):
        vert = Vertex(key)
        self.vertList[key] = vert
        self.numVertices = self.numVertices+1
        
    def addEdge(self, f, t, cost=0):
        if f not in self.vertList:
            self.addVertex(f)
        if t not in self.vertList:
            self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], cost)
        
    def getVertex(self, n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None
        
    def getVertices(self):
        return self.vertList.keys()

## 3.宽度优先搜索

词梯问题

### 3.1创建词桶

In [3]:
def buildGraph(wordFile):
    d = {}
    graph = Graph()
    #构建词桶
    with open(wordFile, "r") as f:
        for line in f:
            word = line[:-1]
            for i in range(len(word)):
                bucket = word[0:i]+"_"+word[i+1:]
                if bucket in d:
                    d[bucket].append(word)
                else:
                    d[bucket] = [word]
    #构建词图
    for bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    graph.addEdge(word1, word2)
    return graph

In [4]:
g = buildGraph("../data/wordfile.txt")

In [5]:
import queue
def BFS(G, start):
    start = G.getVertex(start)
    #设置原点参数
    start.setPreNode(None)
    #设置节点距原点的距离
    start.setDistance(0)
    #节点队列
    vertQueue = queue.Queue()
    vertQueue.put(start)
    while(vertQueue.qsize()>0):
        currentVert = vertQueue.get()
        print(currentVert.key)
        for vert in currentVert.getConnections():
            if vert.getColor() == "white":
                vert.setColor("gray")
                vert.setPreNode(currentVert)
                vert.setDistance(currentVert.getDistance()+1)
                vertQueue.put(vert)
        currentVert.setColor("black")

In [6]:
def DFS(G, start):
    start = G.getVertex(start)
    stack = [start]
    while len(stack)>0:
        currentVert = stack.pop()
        print(currentVert.key)
        for vert in currentVert.getConnections():
            if vert.getColor() == "white":
                vert.setColor("gray")
                vert.setPreNode(currentVert)
                vert.setDistance(currentVert.getDistance()+1)
                stack.append(vert)
        currentVert.setColor("black")

In [7]:
BFS(g, "pool")

pool
poll
cool
fool
pall
pole
foul
foil
fall
pale
pope
fail
sale
sage


In [12]:
DFS(g, "pool")

pool
pollconnectedTo['pall', 'poll', 'pole', 'pool']
poolconnectedTo['poll', 'pool', 'cool', 'fool']
coolconnectedTo['pool', 'cool', 'fool']
foolconnectedTo['pool', 'cool', 'fool', 'foul', 'foil']


In [9]:
#测试
node = g.getVertex("pope")
print(node)
print(node.getDistance())

popeconnectedTo['pole', 'pope']
3


## 4.  骑士周游问题

### 4.1问题建模

In [20]:
#模拟骑士的移动规则
def genLegalMoves(x, y, bdSize):
    newPositions = []
    moveOffsets = [(-1, -2), (-1, 2),(-2, -1),(-2, 1),(1, -2),(1, 2),(2, -1),(2, -1),(2, 1)]
    for i in moveOffsets:
        newx = x + i[0]
        newy = y + i[1]
        if legalPos(newx, bdSize) and legalPos(newy, bdSize):
            newPositions.append((newx, newy))
    return newPositions
    
def legalPos(x, bdSize):
    if x>=0 and x<bdSize:
        return True
    else:
        return False

In [25]:
#用图数据结构对问题建模
def knightGraph(bdSize):
    ktGraph = Graph()
    for i in range(bdSize):
        for j in range(bdSize):
            newPositions = genLegalMoves(i, j, bdSize)
            for p in newPositions:
                ktGraph.addEdge((i,j), p)
    return ktGraph

In [26]:
ktGraph = knightGraph(8)

In [28]:
#测试
node = ktGraph.getVertex((3,3))
print(node)

(3, 3)connectedTo[(2, 1), (2, 5), (1, 2), (1, 4), (4, 1), (4, 5), (5, 2), (5, 4)]


### 4.2 深度优先搜索实现骑士周游

In [ ]:
def knightTour():
    